In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv("../data/process/train_folds.csv")
df_test = pd.read_csv("../data/original/test.csv")

df1 = pd.read_csv("../data/process/train_pred_1.csv")
df1.columns = ["id", "pred_1"]
df2 = pd.read_csv("../data/process/train_pred_2.csv")
df2.columns = ["id", "pred_2"]
df3 = pd.read_csv("../data/process/train_pred_3.csv")
df3.columns = ["id", "pred_3"]
df4 = pd.read_csv("../data/process/train_pred_4.csv")
df4.columns = ["id", "pred_4"]


df_test1 = pd.read_csv("../data/process/test_pred_1.csv")
df_test1.columns = ["id", "pred_1"]
df_test2 = pd.read_csv("../data/process/test_pred_2.csv")
df_test2.columns = ["id", "pred_2"]
df_test3 = pd.read_csv("../data/process/test_pred_3.csv")
df_test3.columns = ["id", "pred_3"]
df_test4 = pd.read_csv("../data/process/test_pred_4.csv")
df_test4.columns = ["id", "pred_4"]


df = df.merge(df1, on="id", how="left")
df = df.merge(df2, on="id", how="left")
df = df.merge(df3, on="id", how="left")
df = df.merge(df4, on="id", how="left")


df_test = df_test.merge(df_test1, on="id", how="left")
df_test = df_test.merge(df_test2, on="id", how="left")
df_test = df_test.merge(df_test3, on="id", how="left")
df_test = df_test.merge(df_test4, on="id", how="left")


In [3]:
df.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont10,cont11,cont12,cont13,target,kfold,pred_1,pred_2,pred_3,pred_4
0,1,B,B,B,C,B,B,A,E,C,...,0.237281,0.377873,0.322401,0.869850,8.113634,0,8.447721,8.183844,8.288112,8.408095
1,2,B,B,A,A,B,D,A,F,A,...,0.906013,0.921701,0.261975,0.465083,8.481233,2,8.434570,8.479641,8.358649,8.438588
2,3,A,A,A,C,B,D,A,D,A,...,0.748809,0.620126,0.541474,0.763846,8.364351,4,8.219918,8.180162,8.041171,8.155833
3,4,B,B,A,C,B,D,A,E,C,...,0.346010,0.714610,0.540150,0.280682,8.049253,3,8.353198,8.272754,8.255105,8.327820
4,6,A,A,A,C,B,D,A,E,A,...,1.000773,0.776742,0.625849,0.250823,7.972260,1,8.258547,8.318773,8.243391,8.238974


In [4]:
sample_submission = pd.read_csv("../data/original/sample_submission.csv")
useful_features = ["pred_1", "pred_2", "pred_3", "pred_4"]
df_test = df_test[useful_features]

final_test_predictions = []
final_valid_predictions = {}
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    valid_ids = xvalid.id.values.tolist()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    

    params = {
        'random_state': 1, 
        'booster': 'gbtree',
        'n_estimators': 7000,
        'learning_rate': 0.03,
        'max_depth': 2
    }
    
    model = XGBRegressor(
        n_jobs=4,
        **params
    )
    model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_1"]
final_valid_predictions.to_csv("level1_train_pred_1.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_1"]
sample_submission.to_csv("level1_test_pred_1.csv", index=False)

[0]	validation_0-rmse:7.54855
[586]	validation_0-rmse:0.71559
0 0.7155161462632544
[0]	validation_0-rmse:7.54526
[542]	validation_0-rmse:0.71565
1 0.715610843791938
[0]	validation_0-rmse:7.54303
[534]	validation_0-rmse:0.71755
2 0.717476602833448
[0]	validation_0-rmse:7.54531
[589]	validation_0-rmse:0.71741
3 0.717380483954798
[0]	validation_0-rmse:7.55076
[634]	validation_0-rmse:0.71560
4 0.715573663436264
0.7163115480559406 0.0009130280148838955


In [5]:
sample_submission = pd.read_csv("../data/original/sample_submission.csv")
useful_features = ["pred_1", "pred_2", "pred_3", "pred_4"]
df_test = df_test[useful_features]

final_test_predictions = []
final_valid_predictions = {}
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    valid_ids = xvalid.id.values.tolist()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    model = RandomForestRegressor(n_estimators=500, n_jobs=-1, max_depth=3)
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_2"]
final_valid_predictions.to_csv("level1_train_pred_2.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_2"]
sample_submission.to_csv("level1_test_pred_2.csv", index=False)

0 0.7158918404838485
1 0.7158796328520922
2 0.717762608455356
3 0.7177572967591042
4 0.7158315757335943
0.716624590856799 0.00092723947707262


In [6]:
sample_submission = pd.read_csv("../data/original/sample_submission.csv")
useful_features = ["pred_1", "pred_2", "pred_3", "pred_4"]
df_test = df_test[useful_features]

final_test_predictions = []
final_valid_predictions = {}
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    valid_ids = xvalid.id.values.tolist()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    model = GradientBoostingRegressor(n_estimators=500, max_depth=3)
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_test_predictions.append(test_preds)
    final_valid_predictions.update(dict(zip(valid_ids, preds_valid)))
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))
final_valid_predictions = pd.DataFrame.from_dict(final_valid_predictions, orient="index").reset_index()
final_valid_predictions.columns = ["id", "pred_3"]
final_valid_predictions.to_csv("level1_train_pred_3.csv", index=False)

sample_submission.target = np.mean(np.column_stack(final_test_predictions), axis=1)
sample_submission.columns = ["id", "pred_3"]
sample_submission.to_csv("level1_test_pred_3.csv", index=False)

0 0.7161197703265872
1 0.7161732393787752
2 0.7181480954219268
3 0.7180441687724307
4 0.7161266100748528
0.7169223767949144 0.0009591070928153001


In [7]:
df = pd.read_csv("../data/process/train_folds.csv")
df_test = pd.read_csv("../data/original/test.csv")
sample_submission = pd.read_csv("../data/original/sample_submission.csv")

df1 = pd.read_csv("level1_train_pred_1.csv")
df2 = pd.read_csv("level1_train_pred_2.csv")
df3 = pd.read_csv("level1_train_pred_3.csv")

df_test1 = pd.read_csv("level1_test_pred_1.csv")
df_test2 = pd.read_csv("level1_test_pred_2.csv")
df_test3 = pd.read_csv("level1_test_pred_3.csv")

df = df.merge(df1, on="id", how="left")
df = df.merge(df2, on="id", how="left")
df = df.merge(df3, on="id", how="left")

df_test = df_test.merge(df_test1, on="id", how="left")
df_test = df_test.merge(df_test2, on="id", how="left")
df_test = df_test.merge(df_test3, on="id", how="left")

df.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont9,cont10,cont11,cont12,cont13,target,kfold,pred_1,pred_2,pred_3
0,1,B,B,B,C,B,B,A,E,C,...,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634,0,8.453034,8.455329,8.459385
1,2,B,B,A,A,B,D,A,F,A,...,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233,2,8.449057,8.460869,8.453468
2,3,A,A,A,C,B,D,A,D,A,...,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351,4,8.233142,8.239456,8.215027
3,4,B,B,A,C,B,D,A,E,C,...,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253,3,8.369102,8.343738,8.377245
4,6,A,A,A,C,B,D,A,E,A,...,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260,1,8.257131,8.259919,8.264252


In [8]:
useful_features = ["pred_1", "pred_2", "pred_3"]
df_test = df_test[useful_features]

final_predictions = []
scores = []
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    model = LinearRegression()
    model.fit(xtrain, ytrain)
    
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold, rmse)
    scores.append(rmse)

print(np.mean(scores), np.std(scores))

0 0.7155189563692765
1 0.7156079344795435
2 0.7174787863004215
3 0.7173881074935493
4 0.7155493504649458
0.7163086270215473 0.0009193042471620706


In [9]:
sample_submission.target = np.mean(np.column_stack(final_predictions), axis=1)
sample_submission.to_csv("../data/submit/013_submission.csv", index=False)

score:0.71729